## Testing on Data
Opening the database files


~corpus folder

In [1]:
import os
import sys
import json

sys.path.append(os.getcwd())
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from dataloc import directory_path
from pathlib import Path

data_path = Path(directory_path)

print(data_path)

# full_path = os.path.join(directory_path, r'corpus\document2anumber.json')
# full_path = directory_path + r"\corpus\document2anumber.json"
full_path = data_path / "corpus" / "document2anumber.json"

print(full_path)

with open(full_path, 'r') as file:
    data = json.load(file)

..\..\wamex\data
..\..\wamex\data\corpus\document2anumber.json


# Open Domain Dictionary Files

In [3]:
Domain_Dictionary_Path = {"geological_timescales": "Domain_Dictionary/geological_timescales.txt",
                          "locations": "Domain_Dictionary/locations.txt",
                          "minerals": "Domain_Dictionary/minerals.txt",
                          "ores_deposits": "Domain_Dictionary/ores_deposits.txt",
                          "rocks": "Domain_Dictionary/rocks.txt",
                          "stratigraphy": "Domain_Dictionary/stratigraphy.txt"}

Domain_Dictionary = {}

for domain, path in Domain_Dictionary_Path.items():
    with open(path, "r") as file:
        info = file.read().splitlines()
        if domain == "geological_timescales":
            info = [i.lstrip().split()[0] for i in info]
        info = [i for i in info if i.strip()] # Make sure there are no empty strings
        Domain_Dictionary[domain] = info

        print(info)
        print(len(info))

# print(Domain_Dictionary["minerals"])

['Aalenian', 'Abereiddian', 'Acadian', 'Actonian', 'Adelaidean', 'Aegean', 'Aeronian', 'Aksayan', 'Aktastinian', 'Alaunian', 'Albertan', 'Albian', 'Aldingian', 'Alexandrian', 'Alportian', 'Altonian', 'Amgan', 'Animikean', 'Anisian', 'Aphebian', 'Aptian', 'Aquitanian', 'Aratauran', 'Archean', 'Archeozoic', 'Arenig', 'Arenigian', 'Arikareean', 'Aritan', 'Arnsbergian', 'Arowhanan', 'Artinskian', 'Arundian', 'Asbian', 'Ashgill', 'Asselian', 'Astian', 'Atdabanian', 'Atokan/Derryan', 'Atokan', 'Aurelucian', 'Austinian', 'Auversian', 'Awamoan', 'Ayusokkanian', 'Azoic', 'Baigendzinian', 'Bairnsdalian', 'Baishaean', 'Bajocian', 'Bala', 'Balan', 'Balcombian', 'Bananian', 'Baotan', 'Barremian', 'Barstovian', 'Bartonian', 'Bashkirian', 'Basin', 'Batesfordian', 'Bathonian', 'Batyrbayan', 'Begudian', 'Bendigonian', 'Berriasian', 'Bithynian', 'Black', 'Blackriveran', 'Blackriverian', 'Blancan', 'Bolindian', 'Bolsovian', 'Boomerangian', 'Bortonian', 'Botomian', 'Braxtonian', 'Bridgerian', 'Brigantian'

In [4]:
entity_type = {"geological_timescales": "TIMESCALE",
               "locations": "LOCATION",
               "minerals": "MINERAL",
               "ores_deposits": "ORE_DEPOSIT",
               "rocks": "ROCK",
               "stratigraphy": "STRAT"}

domain_training_data = []
for domain in Domain_Dictionary:
    for key in Domain_Dictionary[domain]:
        input = key.split(" ")
        labels = [f"I-{entity_type[domain]}"] * len(input)
        labels[0] = f"B-{entity_type[domain]}"
        domain_training_data.append({"output":input, "labels":labels})

# with open("NER/Training_data/DomainDictionary.json", "w") as file:
#     json.dump(domain_training_data, file)

### geo ontology
extract info

In [5]:
# https://github.com/CGI-IUGS/timescale-data/tree/master/rdf ontology grabbed here

from rdflib import Graph

# Create a graph
g = Graph()

# Parse the data from the TTL file
g.parse("Domain_Dictionary/isc2020.ttl", format="ttl")

geo_subject_label_vocab = {}

query = """
PREFIX gts: <http://resource.geosciml.org/ontology/timescale/gts#>
PREFIX isc: <http://resource.geosciml.org/classifier/ics/ischart/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?subject ?label
WHERE {
  ?subject a ?type ;
            rdfs:label ?label ;
  FILTER (?type IN (
    gts:Super-Eon,
    gts:Eon, 
    gts:Era, 
    gts:Period, 
    gts:Sub-Period,
    gts:Epoch, 
    gts:Age
  ))
}
"""

# Execute the query
results = g.query(query)

# Process the results
for row in results:
    subject = row.subject
    label = row.label

    geo_subject_label_vocab[subject] = label
    

# Define the query to fetch all subjects and their labels based on [Eon, Era, Period, Epoch, Age]
geo_dict_scale = {}
time_periods = ["Eon", "Era", "Sub-Period", "Period", "Epoch", "Age"]

for time_period_type in time_periods:
    query = f"""
    PREFIX gts: <http://resource.geosciml.org/ontology/timescale/gts#>
    PREFIX isc: <http://resource.geosciml.org/classifier/ics/ischart/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?subject ?label ?broader ?narrower ?comment
    WHERE {{
      ?subject a gts:{time_period_type} ;
                rdfs:label ?label ;
                rdfs:comment ?comment .
        OPTIONAL {{ ?subject skos:broader ?broader }} .
        OPTIONAL {{ ?subject skos:narrower ?narrower }} .
    }}
    """
    # Execute the query
    results = g.query(query)

    # Dictionary to hold the combined comments
    combined_dict = {}
    # Process the results
    for row in results:
        subject = row.subject
        label = row.label
        broader = row.broader
        comment = row.comment
        narrower = row.narrower
  
        # Initialize the dictionary entry if it doesn't exist
        if (subject, label) not in combined_dict:
            combined_dict[(subject, label)] = {
                "broader": broader,
                "narrower": [],
                "comments": []
            }
        
        # Append the comment to the dictionary entry
        if comment not in combined_dict[(subject, label)]["comments"]:
            combined_dict[(subject, label)]["comments"].append(comment)

        # Append to narrower list if not already present
        if narrower and narrower not in combined_dict[(subject, label)]["narrower"]:
            combined_dict[(subject, label)]["narrower"].append(narrower)

    # Print the results
    for (subject, label), data in combined_dict.items():
        broader = data["broader"]
        comments = [str(c) for c in data["comments"]]
        
        # print(f"Subject: {subject}, Label: {label}, Broader: {broader}")
        # print(f"Comments: {comments}")
        # print(f"Narrower: {data['narrower']}")

    geo_dict_scale[time_period_type] = combined_dict


In [6]:
def count_numbers(list):
    count = 0
    for item in list:
        try:
            # Attempt to convert the item to a float
            float(item.replace('|', '').replace('+', ''))
            count += 1
        except ValueError:
            pass
    return count

def find_geo_time_bound(timeBounds):
    bound_count = count_numbers(timeBounds)
    if bound_count == 1:
        bound = timeBounds[-2]
        uncertainty = 0
    elif bound_count == 2:
        bound = timeBounds[-3]
        uncertainty = timeBounds[-2]
    else:
        bound = None
        uncertainty = None
    return bound, uncertainty

# Place the geological time scale dictionary into the Domain Dictionary json file format
time_periods = ["Age", "Epoch", "Period", "Sub-Period", "Era", "Eon"]

timescale_sorted = {"Eon": {}, "Era": {}, "Sub-Period": {}, "Period": {}, "Epoch": {}, "Age": {}}
prev_type = None
for time_period_type in time_periods:
    current_list = {}
    for (subject, label), data in geo_dict_scale[time_period_type].items():
        broader = data["broader"]
        comments = [str(c) for c in data["comments"]]

        label = str(label)
        # Get the broader and narrower labels
        broader_label = str(geo_subject_label_vocab[broader]) if broader else None
        narrower = [str(geo_subject_label_vocab[n]) for n in data["narrower"] if n]
        # Find the upper and lower bounds
        older_bound = comments[0].split(" ") if comments else None
        younger_bound = comments[-1].split(" ") if comments else None
        upperbound, upper_uncertainty = find_geo_time_bound(older_bound)
        lowerbound, lower_uncertainty = find_geo_time_bound(younger_bound)

        timescale_sorted[time_period_type][label] = {
            "broader": broader_label,
            "narrower": narrower,
            "upperbound" : upperbound,
            "upper_uncertainty" : upper_uncertainty,
            "lowerbound" : lowerbound,
            "lower_uncertainty" : lower_uncertainty
        }

print(timescale_sorted)

{'Eon': {'Archean Eon': {'broader': 'Precambrian Supereon', 'narrower': ['Eoarchean Era', 'Mesoarchean Era', 'Neoarchean Era', 'Paleoarchean Era'], 'upperbound': '-4000', 'upper_uncertainty': 0, 'lowerbound': '-2500', 'lower_uncertainty': 0}, 'Hadean Eon': {'broader': 'Precambrian Supereon', 'narrower': [], 'upperbound': '-4567', 'upper_uncertainty': '+|-1', 'lowerbound': '-4000', 'lower_uncertainty': 0}, 'Phanerozoic Eon': {'broader': None, 'narrower': ['Cenozoic Era', 'Mesozoic Era', 'Paleozoic Era'], 'upperbound': '-541.0', 'upper_uncertainty': '+|-1.0', 'lowerbound': '-0.0', 'lower_uncertainty': 0}, 'Proterozoic Eon': {'broader': 'Precambrian Supereon', 'narrower': ['Mesoproterozoic Era', 'Neoproterozoic Era', 'Paleoproterozoic Era'], 'upperbound': '-2500', 'upper_uncertainty': 0, 'lowerbound': '-541.0', 'lower_uncertainty': '+|-1.0'}}, 'Era': {'Cenozoic Era': {'broader': 'Phanerozoic Eon', 'narrower': ['Neogene Period', 'Paleogene Period', 'Quaternary Period'], 'upperbound': '-66.

In [7]:
# Not Finished, decided to do different method

# # Place the geological time scale dictionary into the Domain Dictionary json file format
# time_periods = ["Eon", "Era", "Sub-Period", "Period", "Epoch", "Age"]

# timescale_sorted = {}
# previous_list = {}
# for time_period_type in time_periods:
#     current_list = {}
#     for (subject, label), data in geo_dict_scale[time_period_type].items():
#         broader = data["broader"]
#         comments = [str(c) for c in data["comments"]]

#         # print(f"Subject: {subject}, Label: {label}, Broader: {broader}")
#         # print(f"Comments: {comments}")

#         label = str(label)
            
#         if time_period_type != "Eon":
#             broader_label = str(geo_subject_label_vocab[broader])
#             if broader_label not in current_list:
#                 current_list[broader_label] = {
#                     "type": time_period_type,
#                     label: {
#                         "upperbound" : None,
#                         "upper_uncertainty" : 0,
#                         "lowerbound" : None,
#                         "lower_uncertainty" : 0
#                     }
#                 }
#             else:
#                 current_list[broader_label][label] = {
#                     "upperbound" : None,
#                     "upper_uncertainty" : 0,
#                     "lowerbound" : None,
#                     "lower_uncertainty" : 0
#                 }

#             if label in previous_list:
#                 current_list[broader_label][label] = previous_list[label]
#                 print(previous_list[label])

#         else:
#             timescale_sorted

#     if (time_period_type == "Sub-Period"):
#         current_list.update(previous_list)

#     previous_list = current_list
#     print(current_list)

# print(current_list)



In [8]:
# import rdflib

# with open("Domain_Dictionary/geological_timescales.txt", "r") as file:
#     geological_timescale_text = file.read()

# print(geological_timescale_text)
# #  eons eras periods epochs ages 

# # Extract the geological timescales words


# # Extract the geological timescales
# geological_timescales = geological_timescale_text.split("\n")
# geo_temporal_scale = {}
# geo_temporal_scale["eons"] = []


In [9]:
from dataloc import directory_path

data_path = directory_path
print(Path(data_path) / "corpus" / "document2anumber.json")
print(os.path.join(directory_path, r'corpus\document2anumber.json'))

with open(Path(data_path) / "corpus" / "document2anumber.json", 'r') as file:
    # os.path.join(directory_path, r'corpus\document2anumber.json'), 'r') as file:
    data = json.load(file)

with open(Path(data_path) / "corpus" / "document2keywords.json", 'r') as file:
    # os.path.join(directory_path, r'corpus/document2keywords.json'), 'r') as file:
    datafd = json.load(file)

with open(Path(data_path) / "corpus" / "keyword2documents.json", 'r') as file:
    # os.path.join(directory_path, r'corpus/keyword2documents.json'), 'r') as file:
    datakeys = json.load(file)

for i, d in enumerate(data):
    if i > 0:
        break
    print("document2anumber.json:")
    print(d, data[d])

print()

for i, d in enumerate(datafd):
    if i > 0:
        break
    print("document2keywords.json:")
    print(d, datafd[d])

print()

for i, d in enumerate(datakeys):
    if i > 0:
        break
    print("keyword2documents.json:")
    print(d, datakeys[d])

print()

print("~~~~~~'a073533_nf_e09_1004_2006a_12145691.pdf' testing:")
print(data['a073533_nf_e09_1004_2006a_12145691.pdf'])
print(datafd['a073533_nf_e09_1004_2006a_12145691.pdf'])

print()


with open(Path(data_path) / "wamex_metadata" / "073533.json", 'r') as file:
    meta_data = json.load(file)

print("073533.json testing:")
print(meta_data)



..\..\wamex\data\corpus\document2anumber.json
..\..\wamex\data\corpus\document2anumber.json
document2anumber.json:
a087761_e28_1833_2010_16911280.pdf 087761

document2keywords.json:
a087761_e28_1833_2010_16911280.pdf {'laverton-karonie greenstone belt': {'textrank': 0, 'tf_idf': 0.1870187238}, 'felsic schists shales': {'textrank': 0.0322580645, 'tf_idf': 0.1745545671}, 'mt monger gold': {'textrank': 0.0418788908, 'tf_idf': 0.3740374477}, 'for gold': {'textrank': 0.0470761702, 'tf_idf': 0.2100083723}, 'dolerite': {'textrank': 0.0322580645, 'tf_idf': 0.0453549253}, 'nickel': {'textrank': 0.0322580645, 'tf_idf': 0.0429806772}, 'significant gold': {'textrank': 0.0248490117, 'tf_idf': 0.0708358369}, 'flexure within host greenstone sequences': {'textrank': 0.0182462575, 'tf_idf': 0.1798837496}, 'regolith': {'textrank': 0.0322580645, 'tf_idf': 0.0987627594}, 'ppb au': {'textrank': 0.0322580645, 'tf_idf': 0.0575461992}, 'cowarna rocks project': {'textrank': 0.0226372383, 'tf_idf': 0.4128977757

### Opening PDF 

In [10]:
# ! pip install pypdf
# https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
from pypdf import PdfReader 

reader = PdfReader(Path(directory_path) / "wamex_pdf" / "a073533_nf_e09_1004_2006a_12145691.pdf")
    # os.path.join(directory_path, r'wamex_pdf/a073533_nf_e09_1004_2006a_12145691.pdf'))

print(len(reader.pages))

for i in range(len(reader.pages)):
        print(f"Page {i+1}:")
        print(reader.pages[i].extract_text())


23
Page 1:
  
 
 
 
 
 
32 Kings Park Rd West Perth 6005 
 
 
Midwest Corporation Limited 
 
New Forest Project 
Annual Report for E70 / 1004 
 
26 September 2005 – 25 September 2006 
 
 
 
 
 
 
 
 
 
Author:  David Broomfield  
B.Sc. (Hons), Dip. Ed. 
Validated: David Broomfield 
B.Sc. (Hons), Dip. Ed. 
Date:  20th October 2006 
Page 2:
Midwest Corporation Limited 
Annual Report E09/1004 September 2006  
 
 
 
 
 
 
TABLE OF CONTENTS 
                                   PAGE 
EXECUTIVE SUMMARY.....................................................................................................................1 
1.0 BIBLIOGRAPHIC DATA SHEET..........................................................................................2 
2.0 INTRODUCTION......................................................................................................................3 
2.1 NEW FOREST LOCATION AND ACCESS.........................................................................................

In [11]:
# ! python -m spacy download en_core_web_sm
# ! pip install spacy

In [12]:
import re
import spacy

# Spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# # nlp.Defaults.stop_words = 
# print(nlp.Defaults.stop_words)
# for word in nlp.Defaults.stop_words:
#     print(word)


# Perform Stopwords Removal, Lemmatisation and Tokenisation
def spacy_processing(x):
    doc = nlp(x)
    x = [word.lemma_ for word in doc] # if not word.is_stop]
    return x

# Function to preprocess the data
def preprocess_data(sentence_list):
    output_list = []
    for sentence in sentence_list:
        sentence = sentence.lower()                     # Case folding
        sentence = re.sub(r'[^\w\s]', ' ', sentence)    # Remove punctuation
        sentence = re.sub(r"\s+", " ", sentence)        # Remove extra spaces
        tokens = spacy_processing(sentence)             # Remove stopwords / Lemmatisation / Tokenise
        output_list.append(tokens)
    return output_list

In [13]:
pages = []

for i in range(len(reader.pages)):
    pages.append(reader.pages[i].extract_text())

output = preprocess_data(pages)
for i in range(len(output)):
    print(f"Page {i+1}:")
    print(output[i])

Page 1:
[' ', '32', 'king', 'park', 'rd', 'west', 'perth', '6005', 'midwest', 'corporation', 'limit', 'new', 'forest', 'project', 'annual', 'report', 'for', 'e70', '1004', '26', 'september', '2005', '25', 'september', '2006', 'author', 'david', 'broomfield', 'b', 'sc', 'hon', 'dip', 'ed', 'validate', 'david', 'broomfield', 'b', 'sc', 'hon', 'dip', 'ed', 'date', '20th', 'october', '2006']
Page 2:
['midwest', 'corporation', 'limit', 'annual', 'report', 'e09', '1004', 'september', '2006', 'table', 'of', 'content', 'page', 'executive', 'summary', '1', '1', '0', 'bibliographic', 'datum', 'sheet', '2', '2', '0', 'introduction', '3', '2', '1', 'new', 'forest', 'location', 'and', 'access', '3', '2', '2', 'new', 'forest', 'tenure', '5', '3', '0', 'new', 'forest', 'project', 'history', '8', '4', '0', 'geology', '9', '4', '1', 'regional', 'geology', 'of', 'the', 'melia', 'creek', 'tenement', '9', '4', '2', 'structure', 'and', 'metamorphism', '9', '4', '3', 'iron', 'mineralisation', '10', '4', '4'

In [14]:
print(pages[2].split()[177])

banded


In [15]:
import re
import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Perform Stopwords Removal, Lemmatisation and Tokenisation
def spacy_processing(x):
    doc = nlp(x)
    x = [word.lemma_ for word in doc] # if not word.is_stop]
    return x

# Function to preprocess the data
def preprocess_data(sentence_list):
    output_list = []
    for sentence in sentence_list:
        sentence = sentence.lower()                     # Case folding
        sentence = re.sub(r'[^\w\s]', ' ', sentence)    # Remove punctuation
        sentence = re.sub(r"\s+", " ", sentence)        # Remove extra spaces
        tokens = spacy_processing(sentence)             # Lemmatisation / Tokenise
        output_list.append(tokens)
    return output_list

def preprocess_word(word):
    word = word.lower()                     # Case folding
    word = re.sub(r'[^\w\s]', ' ', word)    # Remove punctuation
    word = re.sub(r"\s+", " ", word)        # Remove extra spaces
    return word

def preprocess_text(text):
    text = " ".join([token.text for token in nlp(text)])
    return text

In [16]:
# ! python -m spacy download en_core_web_sm

In [17]:
import spacy
import re

# Load the spaCy model
sp_sm = spacy.load('en_core_web_sm')

def spacy_large_ner(document):
    # Extract named entities using spaCy
    entities = {(ent.text.strip(), ent.label_) for ent in sp_sm(document).ents}
    
    # Extract geological timescales
    pattern = r'\b\d+(?:[\.,]\d+)?\s*(?:Ma|ka|Ga|MYA|KYA)\b'
    matches = re.finditer(pattern, document, re.IGNORECASE)
    geological_timescales = {(match.group().strip(), 'GEOLOGICAL_TIME') for match in matches}
    
    # Combine geological timescales with named entities
    all_entities = entities.union(geological_timescales)
    
    return all_entities

# Sample text from geological surveys
text = """
This report has been prepared as an investigation of the Mt Aubrey tenement, as part of Midwest’s New
Forest project in the Murchison Region of Western Australia. The report is presented as an Annual Report
to be submitted to the Department of Industry and Resources as part of the conditions of the granting of
E09/1004 and covers the period from the 26 September 2005 to the 25 September 2006. We also have dates like
50 Ma and 2000 ka which are geological timescales.
"""

# Extract entities and geological timescales
print(spacy_large_ner(text))



{('50 Ma', 'GEOLOGICAL_TIME'), ('the Murchison Region of Western Australia', 'LOC'), ('2000 ka', 'GEOLOGICAL_TIME'), ('the 25 September 2006', 'DATE'), ('the 26 September 2005', 'DATE'), ('Midwest', 'LOC'), ('50', 'CARDINAL'), ('the Department of Industry and Resources', 'ORG'), ('2000', 'DATE'), ('New\nForest', 'ORG')}


In [18]:
import spacy
import re
from dateutil import parser

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def format_ner_output(text):
    # Tokenize the text using spaCy
    doc = nlp(text)
    
    # Define the geological timescales pattern
    timescale_pattern = r'\b\d+(?:[\.,]\d+)?\s*(?:Ma|ka|Ga|MYA|KYA)\b'
    
    # Extract geological timescales and their positions
    geological_timescales = re.finditer(timescale_pattern, text, re.IGNORECASE)
    
    # Create a dictionary to store geological timescale spans
    geo_timescale_spans = set()

    for match in geological_timescales:
        start, end = match.span()
        geo_timescale_spans.add((start, end))

    # print(geo_timescale_spans)
    
    # Prepare to format the output
    tokens_with_labels = []

    # Process tokens and assign labels
    for token in doc:
        token_text = token.text
        token_start = token.idx
        token_end = token_start + len(token_text)
        
        # Check if token is part of a geological timescale
        token_label = 'O'

        # Check if token is a recognized entity
        if token.ent_type_:
            token_label = token.ent_type_
        

        for start, end in geo_timescale_spans:
            if token_start >= start and token_end <= end:
                if (token_start == start):
                    token_label = f'B-GEO_TIME'
                else:
                    token_label = f'I-GEO_TIME'
                break
        
        tokens_with_labels.append(f"{token_text} {token_label}")
    
    return tokens_with_labels


# def extract_relationship_entities(doc):
#     relationships = []
    
#     for token in doc:
#         if token.dep_ in ('attr', 'dobj', 'nsubj'):
#             head = token.head
#             if head.dep_ in ('ROOT', 'attr', 'dobj'):
#                 relationship = {
#                     'entity': token.text,
#                     'relationship': head.text
#                 }
#                 relationships.append(relationship)
    
#     return relationships

# Sample text from geological surveys
text = "Mapping and geochronology by the Geological Society of Australia ( Arriens , 1971 ) reveal that the granitic rocks in the western part of the Yalgoo 1:250,000 map sheet are in the order of 2,800 to 3,000 ma ."

# Format the NER output
ner = format_ner_output(text)
print(ner)

# # Extract temporal information
# doc = nlp(text)

# # Extract relationship entities
# relationships = extract_relationship_entities(doc)
# print("\nRelationship Entities:")
# for rel in relationships:
#     print(f"Entity: {rel['entity']}, Relationship: {rel['relationship']}")

['Mapping O', 'and O', 'geochronology O', 'by O', 'the ORG', 'Geological ORG', 'Society ORG', 'of ORG', 'Australia ORG', '( O', 'Arriens GPE', ', O', '1971 DATE', ') O', 'reveal O', 'that O', 'the O', 'granitic O', 'rocks O', 'in O', 'the O', 'western O', 'part O', 'of O', 'the O', 'Yalgoo O', '1:250,000 O', 'map O', 'sheet O', 'are O', 'in O', 'the O', 'order O', 'of O', '2,800 CARDINAL', 'to CARDINAL', '3,000 B-GEO_TIME', 'ma I-GEO_TIME', '. O']


### Sort Data

In [19]:
filepath = Path(directory_path) / "wamex_xml"
content = {}
for root, dirs, files in os.walk(filepath):
    for file in files:
        try:
            with open(filepath / file, 'r') as f:
                content[file] = json.load(f)
        except:
            pass

In [20]:
for a in content:
    for b in content[a]:
        entities = spacy_large_ner(b)
        contains_geological_time = any(label == 'GEOLOGICAL_TIME' for _, label in entities)
        if contains_geological_time:
            print('"',b.strip(),'"')
            # print(entities)
        


" Mapping and geochronology by the Geological Society of Australia (Arriens, 1971) reveal that the granitic rocks in the western part of the Yalgoo 1:250,000 map sheet are in the order of 2,800 to 3,000 ma. "
" After the Mangaroon Orogeny had deformed and metamorphosed the Gascoyne Complex around 1620Ma, the Bangemall Basin comprising the Edmund and Collier sub-basins, developed between the Pilbara and Yilgarn Cratons. "
" It is younger than 1620Ma and older than the ca. "
" 1465Ma suite of dolerite sills that intrude it. "
" Most of the Collier Basin developed to the north and east, outside the Project area at about 1400 to 1070Ma. "
" A second system of dolerite sills intrudes both the Edmund and Collier Group rocks and is dated at 1078 to 1070Ma. "
" The 1070-755Ma Edmundian Orogeny was an intracratonic event that deformed the Bangemall Supergroup, reactivating basement structures as reverse faults. "
" Around 570Ma the Mulka Orogeny further deformed the rocks and saw intrusion of d

In [21]:
def clean(text):
    if text[-3:] == "Mt.":
        text = text[:-3] + "Mt ."
    else:
        text = text.replace("Mt.", "Mt")
    return text

proprocess_content = content.copy()

for a in proprocess_content:
    for i, b in enumerate(proprocess_content[a]):
        proprocess_content[a][i] = preprocess_text(clean(b))

### LOAD NER AND RUN

In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

save_directory = "./models/saved_model"
# Load the tokenizer and model from the saved directory
# tokenizer = AutoTokenizer.from_pretrained(save_directory)
# model = AutoModelForSeq2SeqLM.from_pretrained(save_directory).to(device)

def tokenize_data(texts, tokenizer, max_length=256):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

def predict_entities(texts, model, tokenizer):
    inputs = tokenize_data(texts, tokenizer)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation
        outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# # Predict geological entities
# predictions = predict_entities(geo_texts, model, tokenizer)

# # Print the results
# print(predictions)
# for text, prediction in zip(geo_texts, predictions):
#     print(f"Text: {text}")
#     print(f"Prediction: {prediction}")
#     print()

In [23]:
def NER_tagging(content, model, tokenizer):
    ner_tagged = {}
    for a in content:
        ner_tagged[a] = {}
        for b in content[a]:
            ner_tagged[a][b] = predict_entities(b, model, tokenizer)

    return ner_tagged

# Perform NER tagging
# ner_tagged = NER_tagging(proprocess_content, model, tokenizer)

In [24]:
# Write to JSON file
# with open("Results/NER_tagged.json", "w") as json_file:
#     json.dump(ner_tagged, json_file)

# Load the NER tagged data
with open("Results/NER_tagged.json", 'r') as file:
    ner_tagged = json.load(file)

In [25]:
from datetime import datetime
import re

def format_date_string(text):
    # Regular expression to match:
    # 1. Month Day , Year (with extra space before the comma)
    # 2. Month Day,Year (without space after the comma)
    pattern = re.compile(r"(\b\w+\s\d{1,2})\s?,\s?(\d{4})")
    # Replace the pattern with "Month Day, Year" with the correct spacing
    formatted_text = pattern.sub(r"\1, \2", text)
    return formatted_text

def find_date_pattern(text):
    # Regular expression for matching the pattern dd/mm/yyyy
    date_pattern = re.compile(r"(0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|1[0-2])/(?:[0-9]{2}|[0-9]{4})")
    
    # Search for the pattern in the text
    match = date_pattern.search(text)
    
    if match:
        return match.group(0)
    return None

# Convert the day into an ordinal number (1st, 2nd, 3rd, etc.)
def ordinal(n):
    return "%d%s" % (n, "th" if 11 <= n <= 13 else {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th"))

def parse_date(found_date):
    # Determine if the year is two or four digits
    if len(found_date.split('/')[-1]) == 2:
        # Assume that two-digit years belong to the 2000s
        date_obj = datetime.strptime(found_date, "%d/%m/%y")
    else:
        date_obj = datetime.strptime(found_date, "%d/%m/%Y")
    
    return date_obj

# ________________________________________________________________________  


def find_month_year_pattern(text):
    # Regular expression for matching the pattern MM/YYYY (ensures month is two digits)
    month_year_pattern = re.compile(r"(0[1-9]|1[0-2])/\d{4}")
    
    # Search for the pattern in the text
    match = month_year_pattern.search(text)
    
    if match:
        return match.group(0)
    return None

def format_month_year(month_year):
    # Parse the date string
    date_obj = datetime.strptime(month_year, "%m/%Y")
    # Format as "Month Year"
    return date_obj.strftime("%B %Y")


# ________________________________________________________________________  

# Example usage
text = "This report was created on 14/12/2001 for the project."
found_date = find_date_pattern(text)

if found_date:
    print(f"Found date: {found_date}")
else:
    print("No date pattern found.")

time_tagged = ner_tagged
for a in time_tagged:

    for text in time_tagged[a]:
        curr = format_date_string(text).replace(" , ", ", ")
        time_tagged[a][text] = curr
        found_date = find_date_pattern(text)
        while found_date:
            # print(found_date)
            # Format the date as "14th December 2001"
            date_obj = parse_date(found_date)
            formatted_date = f"{ordinal(date_obj.day)} {date_obj.strftime('%B')} {date_obj.year}"
            # print(text)
            time_tagged[a][text] = curr.replace(found_date, formatted_date)
            # print(time_tagged[a][text])
            found_date = find_date_pattern(time_tagged[a][text])
            curr = time_tagged[a][text]
            
        found_month_year = find_month_year_pattern(text)
        while found_month_year:
            # Format the month and year as "December 2001"
            formatted_month_year = format_month_year(found_month_year)
            time_tagged[a][text] = curr.replace(found_month_year, formatted_month_year)
            found_month_year = find_month_year_pattern(time_tagged[a][text])
            curr = time_tagged[a][text]


Found date: 14/12/20


In [26]:
from transformers import AutoTokenizer, BertForTokenClassification
import torch

from temporal_taggers.evaluation import merge_tokens, insert_tags_in_raw_text


import pdb

def do_nothing():
    pass

pdb.set_trace = do_nothing

def clean_timex_tags(text):
    # Regular expression to find nested TIMEX3 tags
    nested_timex_pattern = re.compile(r'<TIMEX3[^>]*>(<TIMEX3[^>]*>[^<]+</TIMEX3>)</TIMEX3>')
    
    # Replace the nested TIMEX3 tags with a single TIMEX3 tag
    while nested_timex_pattern.search(text):
        text = nested_timex_pattern.sub(r'\1', text)
    
    return text

time_tagged_full = time_tagged

time_model = BertForTokenClassification.from_pretrained("satyaalmasian/temporal_tagger_BERT_tokenclassifier").to(device)
time_tokenizer = AutoTokenizer.from_pretrained("satyaalmasian/temporal_tagger_BERT_tokenclassifier", use_fast=False)

id2label = {v: k for k, v in time_model.config.label2id.items()}
for a in time_tagged_full:
    annotation_id = 1
    for b in time_tagged_full[a]:
        try:
            text = time_tagged_full[a][b]
            print(text)
            processed_text = time_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

            with torch.no_grad():
                result = time_model(**processed_text)

            classification = torch.argmax(result[0], dim=2)

            # Merge the tokens
            merged_tokens = merge_tokens(processed_text["input_ids"][0], classification[0], id2label, time_tokenizer)
            annotated_text, annotation_id = insert_tags_in_raw_text(text, merged_tokens, annotation_id)
            annotated_text = clean_timex_tags(annotated_text)
            time_tagged_full[a][b] = annotated_text
            print(annotated_text)
        except Exception as e:
            print(f"An error occurred while processing the text: {b}")
            continue


Managed By : GME Resources Ltd Level 2 907 Canning Highway Mt .
Managed By : GME Resources Ltd Level 2 907 Canning Highway Mt .
Pleasant WA 6153 Distribution : Department of Industry and Resources - Perth GME Resources Limited NiWest June 2007 1
Pleasant WA 6153 Distribution : Department of Industry and Resources - Perth GME Resources Limited NiWest <TIMEX3 tid="t1" type="DATE" value="">June 2007</TIMEX3> 1
The Eucalyptus Bore Project is located 85 km ENE of Kookynie and 45 km southeast of the Murrin Murrin mine site .
The Eucalyptus Bore Project is located 85 km ENE of Kookynie and 45 km southeast of the Murrin Murrin mine site .
Access is via the Kookynie - Mt .
Access is via the Kookynie - Mt .
Remarkable Road to Yundamindra Station and thence by station tracks and along fence lines .
Remarkable Road to Yundamindra Station and thence by station tracks and along fence lines .
The former Mount Burgess Gold Mining NL haul road bisects the project area and can also be used to access mos

In [27]:
# # Testings

# from transformers import AutoTokenizer, BertForTokenClassification
# import torch

# from temporal_taggers.evaluation import merge_tokens, insert_tags_in_raw_text

# text = "Sample rate 0.1 Seconds ( 10Hz ) "



# time_model = BertForTokenClassification.from_pretrained("satyaalmasian/temporal_tagger_BERT_tokenclassifier")

# time_tokenizer = AutoTokenizer.from_pretrained("satyaalmasian/temporal_tagger_BERT_tokenclassifier", use_fast=False)

# id2label = {v: k for k, v in time_model.config.label2id.items()}
# annotation_id = 1
# print(text)
# processed_text = time_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
# try:
#     with torch.no_grad():
#         result = time_model(**processed_text)

#     classification = torch.argmax(result[0], dim=2)

#     # Merge the tokens
#     merged_tokens = merge_tokens(processed_text["input_ids"][0], classification[0], id2label, time_tokenizer)
#     annotated_text, annotation_id = insert_tags_in_raw_text(text, merged_tokens, annotation_id)
#     print(annotated_text)
# except Exception as e:
#     print(f"An error occurred while processing the text: {text}")
#     print(e)


    

In [28]:
import re

def extract_timex_and_spans(text):
    # Regular expression to find all TIMEX3 tags and extract their content
    timex_pattern = re.compile(r'<TIMEX3[^>]*>([^<]+)</TIMEX3>')
    
    timex_values = []
    spans = []
    types = []
    cleaned_text = text
    match = timex_pattern.search(cleaned_text)
    
    # Find all TIMEX3 tags in the text
    # for match in timex_pattern.finditer(cleaned_text):
    while match:
        timex_value = match.group(1)
        timex_values.append(timex_value)

        type_pattern = re.compile(r'type="([^"]*)"')
        type = type_pattern.search(match.group(0)).group(1)
        types.append(type)
        
        # Calculate the start and end span in the original text
        start_span = match.span()[0]
        end_span = start_span + len(timex_value)
        spans.append((start_span, end_span))
        
        # Update the cleaned text by removing the TIMEX3 tag
        cleaned_text = cleaned_text.replace(match.group(0), timex_value)
        match = timex_pattern.search(cleaned_text)
        
    return timex_values, cleaned_text, spans, types

for a in time_tagged_full:
    for b in time_tagged_full[a]:
        # print(time_tagged_full[a][b])
        if "TIMEX3" in time_tagged_full[a][b]:
            print(time_tagged_full[a][b])
            timex_values, cleaned_text, spans, types = extract_timex_and_spans(time_tagged_full[a][b])
            print(timex_values)
            print(cleaned_text)
            print(spans)
            print(types)
            print()


Pleasant WA 6153 Distribution : Department of Industry and Resources - Perth GME Resources Limited NiWest <TIMEX3 tid="t1" type="DATE" value="">June 2007</TIMEX3> 1
['June 2007']
Pleasant WA 6153 Distribution : Department of Industry and Resources - Perth GME Resources Limited NiWest June 2007 1
[(106, 115)]
['DATE']

NiWest <TIMEX3 tid="t2" type="DATE" value="">June 2007 26</TIMEX3> Soil sampling was undertaken on a one hundred by one hundred metre grid sampling the -80 mesh material 10 centimetres below the surface .
['June 2007 26']
NiWest June 2007 26 Soil sampling was undertaken on a one hundred by one hundred metre grid sampling the -80 mesh material 10 centimetres below the surface .
[(7, 19)]
['DATE']

NiWest has undertaken an Aboriginal Heritage Survey ( Machin and Glendenning <TIMEX3 tid="t3" type="DATE" value="">2002</TIMEX3> ) of the <TIMEX3 tid="t4" type="DATE" value="">former</TIMEX3> tenements that make up the GME Resources Eucalyptus Bore Project area .
['2002', 'former

In [62]:
with open("Results/NER_tagged.json", 'r') as file:
    ner_tagged = json.load(file)

count = 0
error = 0
numOfTemporals = 0

for a in ner_tagged:
    for b in ner_tagged[a]:
        # print(len(b.split(" ")))
        # print(len(ner_tagged[a][b][0].split(" ")))

        # Check if the number of tokens match
        if len(b.split(" ")) != len(ner_tagged[a][b][0].split(" ")):
            error += 1
        else:
            entities = spacy_large_ner(b)

            tag_labels = ner_tagged[a][b][0].split(" ")

            contains_geological_time = any(label == 'GEOLOGICAL_TIME' for _, label in entities)
            contains_real_time = "TIMEX3" in time_tagged_full[a][b]
            if contains_geological_time or contains_real_time:
                numOfTemporals += 1

            #  Add the geological time tags
            if contains_geological_time:
                temporalLabels = format_ner_output(b)
                # print(b)
                # print(temporalLabels)
                
                for i, word in enumerate(temporalLabels):
                    label = word.split(" ")[1]
                    if "GEO_TIME" in label or "DATE" in label:
                        tag_labels[i] = label

            #  Add the real time tags
            if contains_real_time:
                doc = nlp(b)
                timex_values, cleaned_text, spans, types = extract_timex_and_spans(time_tagged_full[a][b])
                idx= 0
                numOfTemporals = len(timex_values)
                temporalNum = 0
                span = spans[temporalNum]
                for i, word in enumerate(doc):
                    endIdx = idx + len(word.text)
                    if idx >= span[0] and endIdx <= span[1]:
                        if idx == span[0]:
                            tag_labels[i] = "B-DATE"
                        else:
                            tag_labels[i] = "I-DATE"
                    idx = endIdx + 1
                    if idx > span[1]:
                        temporalNum += 1
                        if temporalNum < len(spans):
                            span = spans[temporalNum]

            ner_tagged[a][b] = [" ".join(tag_labels)]
            # print(ner_tagged[a][b])
        count += 1
        
print(f"Total number of sentences: {count}")
print(f"Error rate: {error/count}")
print(f"Number of Temporals: {numOfTemporals/count}")


Total number of sentences: 1945
Error rate: 0.3079691516709512
Number of Temporals: 0.002056555269922879


In [61]:
# # Write to JSON file
# with open("Results/Temporal_NER_tagged.json", "w") as json_file:
#     json.dump(ner_tagged, json_file)

# Load the NER tagged data
with open("Results/Temporal_NER_tagged.json", 'r') as file:
    temporal_ner_tagged = json.load(file)

Size of variables
- Content Shape - {filename: [sentence,...]}
- NER_tagging shape - {filename: {sentence: [tagged_sentence,...]}}

In [30]:
# Write to JSON file
# with open("Results/NER_tagged.json", "w") as json_file:
#     json.dump(ner_tagged, json_file)

# Load the NER tagged data
with open("Results/NER_tagged.json", 'r') as file:
    ner_tagged = json.load(file)

### RE

In [31]:
# https://huggingface.co/models?other=relation-extraction

### REBEL Model

In [32]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# https://www.superteams.ai/blog/knowledge-graph-from-unstructured-text
# https://github.com/gswycf/Joint-Extraction-of-Entities-and-Relations-Based-on-a-Novel-Tagging-Scheme
# https://academic.oup.com/jamia/article/20/5/828/727128?login=false
# https://dl.acm.org/doi/fullHtml/10.1145/3462475
# https://huggingface.co/Babelscape/mrebel-large

def extract_triplets_typed(text):
    triplets = []
    relation = ''
    text = text.strip()
    current = 'x'
    subject, relation, object_, object_type, subject_type = '','','','',''

    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").replace("tp_XX", "").replace("__en__", "").split():
        if token == "<triplet>" or token == "<relation>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                relation = ''
            subject = ''
        elif token.startswith("<") and token.endswith(">"):
            if current == 't' or current == 'o':
                current = 's'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                object_ = ''
                subject_type = token[1:-1]
            else:
                current = 'o'
                object_type = token[1:-1]
                relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '' and object_type != '' and subject_type != '':
        triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/mrebel-large", src_lang="en_XX", tgt_lang="tp_XX")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/mrebel-large").to(device)
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 2,
    "num_return_sequences": 2,
    "forced_bos_token_id": None,
}

# # Text to extract triplets from
# text = """
# This report has been prepared as an investigation of the Mt Aubrey tenement, as part of Midwest’s New Forest project in the Murchison Region of Western Australia. The report is presented as an Annual Report to be submitted to the Department of Industry and Resources as part of the conditions of the granting of E09/1004 and covers the period from the 26 September 2005 to the 25 September 2006.
# """
# # Tokenizer text
# model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

# # Generate
# generated_tokens = model.generate(
#     model_inputs["input_ids"].to(model.device),
#     attention_mask=model_inputs["attention_mask"].to(model.device),
#     decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
#     **gen_kwargs,
# )

# # Extract text
# decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

# # Extract triplets
# for idx, sentence in enumerate(decoded_preds):
#     print(f'Prediction triplets sentence {idx}')
#     print(extract_triplets_typed(sentence))


In [33]:

def extract_relationships(content, model, tokenizer):
    relation_extraction = {}
    for c in content:
        relation_extraction[c] = []
        for content_text in content[c]:
            model_inputs = tokenizer(content_text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')
            generated_tokens = model.generate(
                model_inputs["input_ids"].to(model.device),
                attention_mask=model_inputs["attention_mask"].to(model.device),
                decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
                **gen_kwargs,
            )
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
            for extraction in decoded_preds:
                relation_extraction[c].append(extract_triplets_typed(extraction))
    return relation_extraction

# Perform relationship extraction using REBEL
# relation_extraction = extract_relationships(content, model, tokenizer)

In [34]:
# Write to JSON file
# with open("Results/REBEL_results.json", "w") as json_file:
#     json.dump(relation_extraction, json_file)

# # Load the NER tagged data
with open("Results/REBEL_results.json", 'r') as file:
    relation_extraction = json.load(file)

In [35]:
# for c in relation_extraction:
#     print(c)
#     for r in relation_extraction[c]:
#         print(r)
#         for content_text in r:
#             print(content_text)

### Construct Triples

In [36]:
# # for c in relation_extraction:
# #     print(c)
# #     for sentence, relations in zip(ner_tagged[c], relation_extraction[c]):
# #         entities = ner_tagged[c][sentence]
# #         print(sentence)
# #         # print(ner)
# #         print(relation_extraction[c])
# #         doc = nlp(sentence.strip())
# #         doc = [token.text for token in doc]
# #         print(entities)
# #         # print(doc)
# #         print(len(doc))
# #         print(len(entities[0].split(" ")))
# #         # for content_text in r:
# #         #     print(content_text)

# # print(ner_tagged)

# for c in relation_extraction:
#     print(c)
#     for idx, sentence in enumerate(ner_tagged[c]):

#         entities = ner_tagged[c][sentence]
#         r1 = relation_extraction[c][idx*2]
#         r2 = relation_extraction[c][idx*2+1]
#         print("Sentence: ", sentence)
#         print(r1)
#         print(r2)
#         # print(ner)
#         # print(relation_extraction[c])
#         doc = nlp(sentence.strip())
#         doc = [token.text for token in doc]
#         print(entities)
#         # print(doc)
#         print(len(doc))
#         print(len(entities[0].split(" ")))
#         # for content_text in r:
#         #     print(content_text)
#     break

# print(ner_tagged)